In [62]:
# import dependendencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd

In [65]:
df = pd.read_csv('uscities.csv')
df.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276.0,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574.0,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019.0,polygon,False,True,America/New_York,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7936,-96.7662,5743938,1526.0,polygon,False,True,America/Chicago,1,75287 75098 75233 75254 75251 75252 75253 7503...,1840019440


In [80]:
cities = list(df['city'])
state_ids = list(df['state_id'])

In [81]:
mod_cities = []
mod_state_ids = []

for city, state_id in zip(cities, state_ids):
    mod_city = city.replace(' ', '-').lower()
    mod_cities.append(mod_city)
    
    mod_state_id = state_id.lower()
    mod_state_ids.append(mod_state_id)
    

In [86]:
selected_columns = df[['lat', 'lng', 'population']]
uscities_df = selected_columns.copy()
uscities_df['state'] = mod_state_ids
uscities_df['city'] = mod_cities
cols = ['state', 'city', 'lat', 'lng', 'population']
uscities_df = uscities_df[cols]
uscities_df

,state,city,lat,lng,population
0,ny,new-york,40.6943,-73.9249,18713220
1,ca,los-angeles,34.1139,-118.4068,12750807
2,il,chicago,41.8373,-87.6862,8604203
3,fl,miami,25.7839,-80.2102,6445545
4,tx,dallas,32.7936,-96.7662,5743938
...,...,...,...,...,...
28367,ca,poso-park,35.8114,-118.6364,2
28368,ok,lotsee,36.1334,-96.2091,2
28369,mn,the-ranch,47.3198,-95.6952,2
28370,sd,roswell,44.0060,-97.6975,2


In [99]:
filtered_df = uscities_df.loc[(uscities_df['state'] == 'ny') | (uscities_df['state'] == 'nj') | (uscities_df['state'] == 'pa')]
filtered_df

filtered_df = filtered_df.sort_values(by=['state']).reset_index(drop = True)
filtered_df

,state,city,lat,lng,population
0,nj,tavistock,39.8758,-75.0272,5
1,nj,vienna,40.8698,-74.8810,1296
2,nj,butler,40.9989,-74.3471,7654
3,nj,emerson,40.9748,-74.0239,7596
4,nj,bernardsville,40.7268,-74.5918,7594
...,...,...,...,...,...
3033,pa,palmdale,40.2963,-76.6233,1137
3034,pa,bradford-woods,40.6369,-80.0809,1141
3035,pa,castle-shannon,40.3664,-80.0194,8216
3036,pa,franklin,41.3936,-79.8425,8802


In [140]:
states = ['nj', 'ny', 'pa']

state_dfs = []

def create_state_dfs(source_df, states):
   
    for state in states:
        print(state)
        state_df  = source_df.loc[source_df['state'] == state]
        state_df = state_df.sort_values(by=['population'], ascending=False).reset_index(drop = True).head(25)
        state_dfs.append(state_df)
        state_df.empty
        print(state_dfs)


create_state_dfs(filtered_df, states) 
state_dfs

# Concatenate the dataframes
top_cities_df = pd.concat(state_dfs, ignore_index=True)
top_cities_df

nj
[   state           city      lat      lng  population
0     nj        trenton  40.2236 -74.7641      290716
1     nj         newark  40.7245 -74.1725      282011
2     nj    jersey-city  40.7161 -74.0682      262075
3     nj  atlantic-city  39.3797 -74.4527      237054
4     nj       paterson  40.9147 -74.1624      145233
5     nj      elizabeth  40.6657 -74.1912      129216
6     nj       vineland  39.4653 -74.9981       93239
7     nj        clifton  40.8631 -74.1575       85052
8     nj         camden  39.9362 -75.1073       73562
9     nj        passaic  40.8574 -74.1282       69703
10    nj     union-city  40.7674 -74.0323       67982
11    nj        bayonne  40.6661 -74.1141       64897
12    nj    east-orange  40.7651 -74.2117       64367
13    nj  new-brunswick  40.4870 -74.4450       55676
14    nj  west-new-york  40.7856 -74.0093       52723
15    nj        hoboken  40.7453 -74.0279       52677
16    nj    perth-amboy  40.5203 -74.2724       51390
17    nj     plainfield 

,state,city,lat,lng,population
0,nj,trenton,40.2236,-74.7641,290716
1,nj,newark,40.7245,-74.1725,282011
2,nj,jersey-city,40.7161,-74.0682,262075
3,nj,atlantic-city,39.3797,-74.4527,237054
4,nj,paterson,40.9147,-74.1624,145233
...,...,...,...,...,...
70,pa,chambersburg,39.9315,-77.6556,53083
71,pa,levittown,40.1537,-74.8530,51726
72,pa,bloomsburg,41.0027,-76.4561,49840
73,pa,uniontown,39.8994,-79.7244,48137


In [2]:
# Open the browser from which we want to extract 
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = "https://www.meetup.com/find/us--nj--trenton/arts-culture/?eventType=inPerson&distance=tenMiles"
 
browser.visit(url)
html = browser.html

In [48]:
# Create a beautiful soup object that stores the website's html
soup = bs(html, 'html.parser')


In [61]:
categories = soup.find_all('div', 'css-1q80wuk')
categories

categories_lst = []

for category in categories:
    for a in category.find_all('a', href=True):
        link_text = a['href']
        category_link = link_text.split('/')
        categories_lst.append(category_link[3])
    

categories_lst

['outdoors-adventure',
 'tech',
 'parents-family',
 'health-wellness',
 'sports-fitness',
 'education',
 'photography',
 'food',
 'writing',
 'language',
 'music',
 'movements',
 'lgbtq',
 'film',
 'games-sci-fi',
 'beliefs',
 'arts-culture',
 'book-clubs',
 'dancing',
 'pets',
 'hobbies-crafts',
 'fashion-beauty',
 'social',
 'career-business']

In [ ]:
# construct urls

In [4]:
# Scrape all the events
# class = css-1gl3lql
events = soup.find_all('p', class_='css-1jy1jkx')
events
events_arr = []
for event in events:
    events_arr.append(event.text)
    

events_arr

['START THE NEW YEAR WITH ART - Free One Day Pass to Grounds For Sculpture Today ',
 'Secrets of the Battle of Princeton: History and Nature Walk']

In [5]:
# Number of events 
no_of_events = len(events_arr)
print(no_of_events)


2


In [18]:
events_descriptions = soup.find_all('p', class_='css-ycqk9')
events_descriptions

group_names_arr = []
for desc in events_descriptions:
    desc_text = desc.text
    desc_text = desc_text.split(':')[1]
    group_names_arr.append(desc_text)
    
group_names_arr

['Central Jersey 56 Plus', 'Secrets of the Battle of Princeton Tours']

In [19]:
attendees = soup.find_all('div', class_="css-1dwjarq")
attendees

attendees_lst = []
for attendee_count in attendees:
    attendee_count = attendee_count.text
    attendee_count = attendee_count.split(' ')[0]
    attendees_lst.append(attendee_count)
    
attendees_lst

['1', '20']

In [46]:
events = soup.find_all('div', class_='css-11tad7j')
events
event_links = []
for event in events:
    
    for a in event.find_all('a', href=True):
        link_text = a['href']
        event_links.append(link_text)
        
event_links

['https://www.meetup.com/CJ-56plus/events/275689348',
 'https://www.meetup.com/Secrets-of-the-Princeton-Battlefield-Tours/events/272652579']

In [47]:
grouped_lst = []

for event, group_name, attendee_count, event_link in zip(events_arr, group_names_arr, attendees_lst, event_links):
    grouped_lst.append([event, group_name, attendee_count, event_link])
    
grouped_lst

[['START THE NEW YEAR WITH ART - Free One Day Pass to Grounds For Sculpture Today ',
  'Central Jersey 56 Plus',
  '1',
  'https://www.meetup.com/CJ-56plus/events/275689348'],
 ['Secrets of the Battle of Princeton: History and Nature Walk',
  'Secrets of the Battle of Princeton Tours',
  '20',
  'https://www.meetup.com/Secrets-of-the-Princeton-Battlefield-Tours/events/272652579']]

In [7]:
# Load the scraped data into a dataframe